In [ ]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 7.3 MB/s 
     |████████████████████████████████| 115 kB 69.1 MB/s 
     |████████████████████████████████| 212 kB 50.7 MB/s 
     |████████████████████████████████| 182 kB 68.4 MB/s 
     |████████████████████████████████| 127 kB 61.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.9 MB/s 
     |████████████████████████████████| 7.6 MB 61.2 MB/s 


In [ ]:
from datasets import Dataset, load_dataset, load_metric
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import os
import json

Download datasets from hugging face

missing children's book dataset

In [ ]:
ag_dataset_train = load_dataset('ag_news', split='train')
ag_dataset_test = load_dataset('ag_news', split='test')
imdb_dataset_train = load_dataset('imdb', split='train')
imdb_dataset_test = load_dataset('imdb', split='test')
rt_dataset_train = load_dataset('rotten_tomatoes', split='train')
rt_dataset_test = load_dataset('rotten_tomatoes', split='test')

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Dataset rotten_tomatoes downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46. Subsequent calls will reuse this data.


In [ ]:
ag_dataset_train

Dataset({
    features: ['text', 'label'],
    num_rows: 120000
})

Building tokenizer

In [ ]:
print("building tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def tokenize_function(examples):
	return tokenizer(examples["text"], padding="max_length", truncation=True)

tk_imdb_dataset_train = imdb_dataset_train.map(tokenize_function, batched=True)
tk_imdb_dataset_test = imdb_dataset_test.map(tokenize_function, batched=True)
tk_ag_dataset_train = ag_dataset_train.map(tokenize_function, batched = True)
tk_ag_dataset_test = ag_dataset_test.map(tokenize_function, batched=True)
tk_rt_dataset_train = rt_dataset_train.map(tokenize_function, batched=True)
tk_rt_dataset_test = rt_dataset_test.map(tokenize_function, batched=True)

building tokenizer...


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/120 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
tk_ag_dataset_train = tk_ag_dataset_train.shuffle(seed=42)
data = tk_ag_dataset_train.train_test_split(.05)
tk_ag_train = data['train']
tk_ag_eval = data['test']

In [ ]:
tk_ag_eval

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 6000
})

In [ ]:
tk_ag_train

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 114000
})

In [ ]:
tk_ag_dataset_test

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 7600
})

In [ ]:
np.unique(tk_ag_train['label'], return_counts=True)

(array([0, 1, 2, 3]), array([28519, 28456, 28499, 28526]))

In [ ]:
np.unique(tk_ag_eval['label'], return_counts=True)

(array([0, 1, 2, 3]), array([1481, 1544, 1501, 1474]))

In [ ]:
np.unique(tk_ag_dataset_test['label'], return_counts=True)

(array([0, 1, 2, 3]), array([1900, 1900, 1900, 1900]))

Loading distilbert model + running AG News dataset

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", 
                                                           num_labels = 4)
model.cuda()
training_args = TrainingArguments(
	output_dir="/content/drive/My Drive/nlp/model_out", 
	evaluation_strategy="steps", 
	num_train_epochs = 1,
  eval_steps = 1000,
  #gradient_accumulation_steps=4,
	# evaluate_during_training = True, #outdated parameter option, do not use
	save_steps = 1000,
	load_best_model_at_end = True,
	learning_rate = 5e-6#, #defaults to 5e-5
	# report_to="wandb"
	)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier

In [ ]:
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
	logits,labels = eval_pred
	predictions = np.argmax(logits, axis=-1)
	print("Labels:", labels)
	print("Predictions:", predictions)
	# print("Logits:", logits)
	# print("Sizes:", labels.shape, predictions.shape, logits.shape)
	# exit()
	# label = np.array([random.randint(0,1) for _ in range(26112)])
	# preds = np.array([random.randint(0,1) for _ in range(26112)])
	# logits = np.array([[random.random() for _ in range(2)] for _ in range(26112)])
	result = metric.compute(predictions=predictions, references=labels)
	return result

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [ ]:
#model.cuda()
trainer = Trainer(
	model=model,
	args = training_args,
	train_dataset = tk_ag_train,
	eval_dataset = tk_ag_eval,
	tokenizer=tokenizer,
	compute_metrics=compute_metrics
	)

trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 114000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 42750
  Number of trainable parameters = 66956548
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster 

Step,Training Loss,Validation Loss,Accuracy
1000,0.362100,0.336615,0.897833
2000,0.297900,0.350443,0.908833
3000,0.300600,0.305784,0.909833
4000,0.280500,0.279961,0.919000
5000,0.267600,0.269348,0.923667
6000,0.263600,0.256794,0.927833
7000,0.280500,0.241930,0.927500
8000,0.262300,0.239214,0.931333
9000,0.250800,0.240610,0.931167
10000,0.256000,0.241764,0.932167


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 6000
  Batch size = 8


Labels: [3 2 1 ... 1 2 0]
Predictions: [2 3 1 ... 1 3 0]


Saving model checkpoint to /content/drive/My Drive/nlp/model_out/checkpoint-1000
Configuration saved in /content/drive/My Drive/nlp/model_out/checkpoint-1000/config.json
Model weights saved in /content/drive/My Drive/nlp/model_out/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/nlp/model_out/checkpoint-1000/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/nlp/model_out/checkpoint-1000/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 6000
  Batch size = 8


...

Labels: [3 2 1 ... 1 2 0]
Predictions: [3 3 1 ... 1 2 0]


Saving model checkpoint to /content/drive/My Drive/nlp/model_out/checkpoint-14000
Configuration saved in /content/drive/My Drive/nlp/model_out/checkpoint-14000/config.json
Model weights saved in /content/drive/My Drive/nlp/model_out/checkpoint-14000/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/nlp/model_out/checkpoint-14000/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/nlp/model_out/checkpoint-14000/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 6000
  Batch size = 8


Labels: [3 2 1 ... 1 2 0]
Predictions: [3 2 1 ... 1 2 0]


Saving model checkpoint to /content/drive/My Drive/nlp/model_out/checkpoint-15000
Configuration saved in /content/drive/My Drive/nlp/model_out/checkpoint-15000/config.json
Model weights saved in /content/drive/My Drive/nlp/model_out/checkpoint-15000/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/nlp/model_out/checkpoint-15000/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/nlp/model_out/checkpoint-15000/special_tokens_map.json


KeyboardInterrupt: ignored

In [ ]:
trainer.predict(tk_ag_dataset_test)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 7600
  Batch size = 8


Step,Training Loss,Validation Loss,Accuracy
1000,0.362100,0.336615,0.897833
2000,0.297900,0.350443,0.908833
3000,0.300600,0.305784,0.909833
4000,0.280500,0.279961,0.919000
5000,0.267600,0.269348,0.923667
6000,0.263600,0.256794,0.927833
7000,0.280500,0.241930,0.927500
8000,0.262300,0.239214,0.931333
9000,0.250800,0.240610,0.931167
10000,0.256000,0.241764,0.932167


Labels: [2 3 3 ... 1 2 2]
Predictions: [2 3 3 ... 1 2 3]


PredictionOutput(predictions=array([[-0.22773798, -3.7745576 ,  5.1971793 , -2.1066117 ],
       [-1.6668842 , -3.2343109 , -1.4503163 ,  5.0352974 ],
       [-1.0096163 , -3.4213765 , -1.6671764 ,  4.661     ],
       ...,
       [-1.6754481 ,  6.098278  , -2.6903117 , -2.9169073 ],
       [ 1.6750785 , -3.6410406 ,  2.180877  , -1.4244785 ],
       [-2.503387  , -4.118053  ,  1.2770433 ,  3.975561  ]],
      dtype=float32), label_ids=array([2, 3, 3, ..., 1, 2, 2]), metrics={'test_loss': 0.2482970505952835, 'test_accuracy': 0.9356578947368421, 'test_runtime': 135.4879, 'test_samples_per_second': 56.094, 'test_steps_per_second': 7.012})

IMDB Dataset

In [ ]:
tk_imdb_dataset_train = tk_imdb_dataset_train.shuffle(seed=42)
data = tk_imdb_dataset_train.train_test_split(.05)
tk_imdb_train = data['train']
tk_imdb_eval = data['test']

In [ ]:
tk_imdb_train

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 23750
})

In [ ]:
tk_imdb_eval

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1250
})

In [ ]:
tk_imdb_dataset_test

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 25000
})

In [ ]:
np.unique(tk_imdb_train['label'], return_counts=True)

(array([0, 1]), array([11884, 11866]))

In [ ]:
np.unique(tk_imdb_eval['label'], return_counts=True)

(array([0, 1]), array([616, 634]))

In [ ]:
np.unique(tk_imdb_dataset_test['label'], return_counts=True)

(array([0, 1]), array([12500, 12500]))

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", 
                                                           num_labels = 2)
model.cuda()
training_args = TrainingArguments(
	output_dir="/content/drive/My Drive/nlp/model_out", 
	evaluation_strategy="steps", 
	num_train_epochs = 3,
  eval_steps = 1000,
  #gradient_accumulation_steps=4,
	# evaluate_during_training = True, #outdated parameter option, do not use
	save_steps = 1000,
	load_best_model_at_end = True,
	learning_rate = 5e-6#, #defaults to 5e-5
	# report_to="wandb"
	)

metric = load_metric("accuracy")
def compute_metrics(eval_pred):
	logits,labels = eval_pred
	predictions = np.argmax(logits, axis=-1)
	print("Labels:", labels)
	print("Predictions:", predictions)
	# print("Logits:", logits)
	# print("Sizes:", labels.shape, predictions.shape, logits.shape)
	# exit()
	# label = np.array([random.randint(0,1) for _ in range(26112)])
	# preds = np.array([random.randint(0,1) for _ in range(26112)])
	# logits = np.array([[random.random() for _ in range(2)] for _ in range(26112)])
	result = metric.compute(predictions=predictions, references=labels)
	return result

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased were not used when in

In [ ]:
#model.cuda()
trainer = Trainer(
	model=model,
	args = training_args,
	train_dataset = tk_imdb_train,
	eval_dataset = tk_imdb_eval,
	tokenizer=tokenizer,
	compute_metrics=compute_metrics
	)

trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 23750
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 8907
  Number of trainable parameters = 66955010


Step,Training Loss,Validation Loss,Accuracy
1000,0.301800,0.255471,0.908800
2000,0.290300,0.282596,0.902400
3000,0.278300,0.260813,0.918400
4000,0.226900,0.275063,0.923200
5000,0.225500,0.256795,0.928000
6000,0.218000,0.251810,0.931200
7000,0.191000,0.264424,0.931200
8000,0.199900,0.255452,0.935200


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1250
  Batch size = 8


Labels: [1 0 1 ... 1 0 0]
Predictions: [1 0 1 ... 1 0 0]


Saving model checkpoint to /content/drive/My Drive/nlp/model_out/checkpoint-1000
Configuration saved in /content/drive/My Drive/nlp/model_out/checkpoint-1000/config.json
Model weights saved in /content/drive/My Drive/nlp/model_out/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/nlp/model_out/checkpoint-1000/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/nlp/model_out/checkpoint-1000/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1250
  Batch size = 8


...

Labels: [1 0 1 ... 1 0 0]
Predictions: [1 0 1 ... 1 0 0]


Saving model checkpoint to /content/drive/My Drive/nlp/model_out/checkpoint-7000
Configuration saved in /content/drive/My Drive/nlp/model_out/checkpoint-7000/config.json
Model weights saved in /content/drive/My Drive/nlp/model_out/checkpoint-7000/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/nlp/model_out/checkpoint-7000/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/nlp/model_out/checkpoint-7000/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1250
  Batch size = 8


Labels: [1 0 1 ... 1 0 0]
Predictions: [1 0 1 ... 1 0 0]


Saving model checkpoint to /content/drive/My Drive/nlp/model_out/checkpoint-8000
Configuration saved in /content/drive/My Drive/nlp/model_out/checkpoint-8000/config.json
Model weights saved in /content/drive/My Drive/nlp/model_out/checkpoint-8000/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/nlp/model_out/checkpoint-8000/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/nlp/model_out/checkpoint-8000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/drive/My Drive/nlp/model_out/checkpoint-6000 (score: 0.25181031227111816).


TrainOutput(global_step=8907, training_loss=0.2428559178761043, metrics={'train_runtime': 3777.2805, 'train_samples_per_second': 18.863, 'train_steps_per_second': 2.358, 'total_flos': 9438302154240000.0, 'train_loss': 0.2428559178761043, 'epoch': 3.0})

In [ ]:
trainer.predict(tk_imdb_dataset_test)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 25000
  Batch size = 8


Labels: [0 0 0 ... 1 1 1]
Predictions: [0 0 0 ... 1 1 1]


PredictionOutput(predictions=array([[ 2.7942128 , -3.1892905 ],
       [ 2.3838286 , -2.9162552 ],
       [ 2.5628147 , -3.1345773 ],
       ...,
       [-1.0857923 ,  0.18515757],
       [-2.3992088 ,  1.4566284 ],
       [-2.907682  ,  1.88634   ]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 1, 1]), metrics={'test_loss': 0.2746902406215668, 'test_accuracy': 0.92588, 'test_runtime': 447.3697, 'test_samples_per_second': 55.882, 'test_steps_per_second': 6.985})

Rotten Tomatoes Dataset

In [ ]:
tk_rt_dataset_train = tk_rt_dataset_train.shuffle(seed=42)
data = tk_rt_dataset_train.train_test_split(.1)
tk_rt_train = data['train']
tk_rt_eval = data['test']

In [ ]:
tk_rt_train

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 7677
})

In [ ]:
tk_rt_eval

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 853
})

In [ ]:
np.unique(tk_rt_eval['label'], return_counts=True)

(array([0, 1]), array([428, 425]))

In [ ]:
tk_rt_dataset_test

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1066
})

In [ ]:
np.unique(tk_rt_train['label'], return_counts=True)

(array([0, 1]), array([3837, 3840]))

In [ ]:
np.unique(tk_rt_dataset_test['label'], return_counts=True)

(array([0, 1]), array([533, 533]))

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", 
                                                           num_labels = 2)
model.cuda()
training_args = TrainingArguments(
	output_dir="/content/drive/My Drive/nlp/model_out", 
	evaluation_strategy="steps", 
	num_train_epochs = 3,
  eval_steps = 100,
	warmup_steps = 100,
	logging_steps = 100,
  #gradient_accumulation_steps=4,
	# evaluate_during_training = True, #outdated parameter option, do not use
	per_device_train_batch_size = 16,
	save_steps = 500,
	load_best_model_at_end = True,
	learning_rate = 5e-6#, #defaults to 5e-5
	# report_to="wandb"
	)

metric = load_metric("accuracy")
def compute_metrics(eval_pred):
	logits,labels = eval_pred
	predictions = np.argmax(logits, axis=-1)
	print("Labels:", labels)
	print("Predictions:", predictions)
	# print("Logits:", logits)
	# print("Sizes:", labels.shape, predictions.shape, logits.shape)
	# exit()
	# label = np.array([random.randint(0,1) for _ in range(26112)])
	# preds = np.array([random.randint(0,1) for _ in range(26112)])
	# logits = np.array([[random.random() for _ in range(2)] for _ in range(26112)])
	result = metric.compute(predictions=predictions, references=labels)
	return result

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased were not used when in

In [ ]:
#model.cuda()
trainer = Trainer(
	model=model,
	args = training_args,
	train_dataset = tk_rt_train,
	eval_dataset = tk_rt_eval,
	tokenizer=tokenizer,
	compute_metrics=compute_metrics
	)

trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 7677
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1440
  Number of trainable parameters = 66955010


Step,Training Loss,Validation Loss,Accuracy
100,0.686900,0.663494,0.743259
200,0.567500,0.462800,0.794842
300,0.467300,0.405973,0.824150
400,0.414400,0.399674,0.814771
500,0.397900,0.372064,0.834701
600,0.359300,0.367792,0.840563
700,0.345300,0.371224,0.832356
800,0.362400,0.361598,0.840563
900,0.348000,0.369840,0.849941
1000,0.318900,0.365108,0.848769


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


Labels: [0 1 0 0 1 0 0 0 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 0 1 0 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 1 0 1 1 1 0 1 0 0 1 1 1 0 0 0 0 1 1 1 0 1 0 0 0 0 1 0 0 1 1 1 1 1
 1 0 0 0 0 1 0 0 0 0 1 1 0 1 1 1 1 0 1 1 0 0 0 0 1 0 0 0 1 1 1 1 1 0 0 1 0
 1 1 1 0 0 1 0 0 1 1 0 0 0 1 1 1 0 0 1 0 1 1 0 1 1 0 1 1 0 0 0 1 1 1 0 1 0
 1 1 0 1 1 0 1 1 1 0 1 1 0 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 0 0 1 0 1 1
 0 0 0 1 1 0 0 1 1 0 0 1 1 1 0 0 1 0 0 1 0 1 1 0 1 1 1 1 1 1 0 0 1 0 1 0 0
 1 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 1 1 1 0 0 1 1 0 1 1 0 1 0 0
 0 0 1 0 0 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 0 0 0 1 1 1 0 1 1 1 1 0 0 1 1 1
 0 0 0 0 0 1 0 1 1 0 0 0 1 1 1 0 1 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 0 0 0 1 1
 1 0 0 1 0 1 1 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 1 0
 0 1 1 1 0 1 0 0 1 1 1 1 1 0 0 1 1 0 1 0 1 1 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1
 1 1 1 0 0 1 1 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 1 1 1 0
 0 1 0 0 1 0 1 0 0 0 1 1 1 1 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 1 0 0
 0 1 1 1 1 0 0 0 

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


...

Labels: [0 1 0 0 1 0 0 0 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 0 1 0 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 1 0 1 1 1 0 1 0 0 1 1 1 0 0 0 0 1 1 1 0 1 0 0 0 0 1 0 0 1 1 1 1 1
 1 0 0 0 0 1 0 0 0 0 1 1 0 1 1 1 1 0 1 1 0 0 0 0 1 0 0 0 1 1 1 1 1 0 0 1 0
 1 1 1 0 0 1 0 0 1 1 0 0 0 1 1 1 0 0 1 0 1 1 0 1 1 0 1 1 0 0 0 1 1 1 0 1 0
 1 1 0 1 1 0 1 1 1 0 1 1 0 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 0 0 1 0 1 1
 0 0 0 1 1 0 0 1 1 0 0 1 1 1 0 0 1 0 0 1 0 1 1 0 1 1 1 1 1 1 0 0 1 0 1 0 0
 1 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 1 1 1 0 0 1 1 0 1 1 0 1 0 0
 0 0 1 0 0 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 0 0 0 1 1 1 0 1 1 1 1 0 0 1 1 1
 0 0 0 0 0 1 0 1 1 0 0 0 1 1 1 0 1 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 0 0 0 1 1
 1 0 0 1 0 1 1 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 1 0
 0 1 1 1 0 1 0 0 1 1 1 1 1 0 0 1 1 0 1 0 1 1 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1
 1 1 1 0 0 1 1 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 1 1 1 0
 0 1 0 0 1 0 1 0 0 0 1 1 1 1 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 1 0 0
 0 1 1 1 1 0 0 0 

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


Labels: [0 1 0 0 1 0 0 0 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 0 1 0 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 1 0 1 1 1 0 1 0 0 1 1 1 0 0 0 0 1 1 1 0 1 0 0 0 0 1 0 0 1 1 1 1 1
 1 0 0 0 0 1 0 0 0 0 1 1 0 1 1 1 1 0 1 1 0 0 0 0 1 0 0 0 1 1 1 1 1 0 0 1 0
 1 1 1 0 0 1 0 0 1 1 0 0 0 1 1 1 0 0 1 0 1 1 0 1 1 0 1 1 0 0 0 1 1 1 0 1 0
 1 1 0 1 1 0 1 1 1 0 1 1 0 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 0 0 1 0 1 1
 0 0 0 1 1 0 0 1 1 0 0 1 1 1 0 0 1 0 0 1 0 1 1 0 1 1 1 1 1 1 0 0 1 0 1 0 0
 1 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 1 1 1 0 0 1 1 0 1 1 0 1 0 0
 0 0 1 0 0 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 0 0 0 1 1 1 0 1 1 1 1 0 0 1 1 1
 0 0 0 0 0 1 0 1 1 0 0 0 1 1 1 0 1 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 0 0 0 1 1
 1 0 0 1 0 1 1 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 1 0
 0 1 1 1 0 1 0 0 1 1 1 1 1 0 0 1 1 0 1 0 1 1 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1
 1 1 1 0 0 1 1 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 1 1 1 0
 0 1 0 0 1 0 1 0 0 0 1 1 1 1 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 1 0 0
 0 1 1 1 1 0 0 0 



Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/drive/My Drive/nlp/model_out/checkpoint-1000 (score: 0.3651081621646881).


TrainOutput(global_step=1440, training_loss=0.3886825634373559, metrics={'train_runtime': 1259.211, 'train_samples_per_second': 18.29, 'train_steps_per_second': 1.144, 'total_flos': 3050856658446336.0, 'train_loss': 0.3886825634373559, 'epoch': 3.0})

In [ ]:
trainer.predict(tk_rt_dataset_test)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1066
  Batch size = 8


Labels: [1 1 1 ... 0 0 0]
Predictions: [1 1 0 ... 0 0 0]


PredictionOutput(predictions=array([[-1.3272141,  0.9924911],
       [-1.856971 ,  1.5912265],
       [ 1.0681944, -1.1127119],
       ...,
       [ 0.9949597, -1.0307143],
       [ 1.119624 , -1.1439887],
       [ 1.4636197, -1.6080972]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 0]), metrics={'test_loss': 0.400054395198822, 'test_accuracy': 0.8330206378986866, 'test_runtime': 18.3608, 'test_samples_per_second': 58.058, 'test_steps_per_second': 7.298})